In [21]:
from typing import TypedDict, Literal
from dotenv import load_dotenv
from google import genai
from google.genai import types
import json
import os

load_dotenv('C:\My Projects\Health-Navigator\credentials.env')

True

In [ ]:
class MedicalInputCheck(TypedDict):
    image_type: Literal['chest_xray', 'colon_tissue', 'text', 'not_valid_image']

In [ ]:
system_prompt = """
    You are a medical image classifier. Analyze the provided image and its title, then classify it into exactly one category.

    Classification Rules:
    - chest_xray: ONLY if the image is a chest X-ray (frontal or lateral view of thorax). Any other type of X-ray → not_valid_image
    - colon_tissue: ONLY if the image shows colon pathology tissue/histology slides
    - text: If the image primarily contains text (documents, screenshots, diagrams with text, etc.) - regardless of content relevance
    - not_valid_image: Everything else (other medical images, non-medical images, unclear images, other anatomical X-rays, etc.)

    Important:
    - When unsure, default to not_valid_image
    - Consider both image content and title, and take the title as a strong hint towards classification

    Output only one of these classes: 'chest_xray', 'colon_tissue', 'text', 'not_valid_image'
"""

In [7]:
image_title = "tissue"

with open('experiment_images/image_1_label_[1].png', 'rb') as f:
      image_bytes = f.read()

client = genai.Client()

response = client.models.generate_content(
    model='gemini-3-flash-preview',
    contents=[
        types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',
        ),
        f"Image Title: {image_title}"
    ],
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        response_mime_type="application/json",
        response_schema=MedicalInputCheck
    )
)

print(response.text)

{"image_type":"not_valid"}


In [24]:
d = json.loads(response.text)
print(d['image_type'])

not_valid
